In [ ]:
#Support Vector Machines (SVM) are a powerful supervised machine learning technique, especially useful for classification and regression tasks. Below is a step-by-step guide to applying SVM using R with the e1071 package.

In [12]:
# Install the 'e1071' package for SVM
install.packages("e1071")

# If you are working with tidyverse-style data manipulation
install.packages("caret")   # For model training and validation


Warning message:
"package 'e1071' is in use and will not be installed"
Warning message:
"package 'caret' is in use and will not be installed"


In [13]:
# Load necessary libraries
library(e1071)
library(caret)

# Load the iris dataset
data(iris)
head(iris)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa


In [14]:
# Split the dataset into training (70%) and test (30%) sets
set.seed(123)  # For reproducibility
index <- createDataPartition(iris$Species, p = 0.7, list = FALSE)

train_data <- iris[index, ]
test_data <- iris[-index, ]


In [15]:
# Train the SVM model using a linear kernel
svm_model <- svm(Species ~ ., data = train_data, kernel = "linear", cost = 1, 
                 scale = TRUE)

# View the summary of the model
summary(svm_model)



Call:
svm(formula = Species ~ ., data = train_data, kernel = "linear", 
    cost = 1, scale = TRUE)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  1 

Number of Support Vectors:  22

 ( 2 10 10 )


Number of Classes:  3 

Levels: 
 setosa versicolor virginica




In [16]:
# Predict the species for the test data
predictions <- predict(svm_model, test_data)

# Print the predictions
print(predictions)


         1          2          6         16         18         20         22 
    setosa     setosa     setosa     setosa     setosa     setosa     setosa 
        23         34         35         38         39         44         46 
    setosa     setosa     setosa     setosa     setosa     setosa     setosa 
        47         51         53         54         64         72         74 
    setosa versicolor versicolor versicolor versicolor versicolor versicolor 
        78         81         85         87         90         91         94 
versicolor versicolor versicolor versicolor versicolor versicolor versicolor 
        99        100        101        106        109        111        116 
versicolor versicolor  virginica  virginica  virginica  virginica  virginica 
       117        120        124        127        133        134        136 
 virginica  virginica  virginica  virginica  virginica versicolor  virginica 
       137        149        150 
 virginica  virginica  virgini

In [17]:
# Confusion Matrix to evaluate accuracy
conf_matrix <- confusionMatrix(predictions, test_data$Species)
print(conf_matrix)


Confusion Matrix and Statistics

            Reference
Prediction   setosa versicolor virginica
  setosa         15          0         0
  versicolor      0         15         1
  virginica       0          0        14

Overall Statistics
                                          
               Accuracy : 0.9778          
                 95% CI : (0.8823, 0.9994)
    No Information Rate : 0.3333          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.9667          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: setosa Class: versicolor Class: virginica
Sensitivity                 1.0000            1.0000           0.9333
Specificity                 1.0000            0.9667           1.0000
Pos Pred Value              1.0000            0.9375           1.0000
Neg Pred Value              1.0000            1.0000           0.9677
P

In [18]:
# Train using radial basis function (RBF) kernel and tune parameters
tuned_model <- tune.svm(Species ~ ., data = train_data,
                        kernel = "radial",
                        cost = 10^(-1:2), gamma = c(0.1, 0.5, 1))

# View best model parameters
print(tuned_model$best.parameters)

# Train final model with tuned parameters
svm_model_tuned <- svm(Species ~ ., data = train_data, 
                       kernel = "radial", 
                       cost = tuned_model$best.parameters$cost, 
                       gamma = tuned_model$best.parameters$gamma)

# Evaluate on test data
predictions_tuned <- predict(svm_model_tuned, test_data)
conf_matrix_tuned <- confusionMatrix(predictions_tuned, test_data$Species)
print(conf_matrix_tuned)


  gamma cost
5   0.5    1
Confusion Matrix and Statistics

            Reference
Prediction   setosa versicolor virginica
  setosa         15          0         0
  versicolor      0         14         2
  virginica       0          1        13

Overall Statistics
                                         
               Accuracy : 0.9333         
                 95% CI : (0.8173, 0.986)
    No Information Rate : 0.3333         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.9            
                                         
 Mcnemar's Test P-Value : NA             

Statistics by Class:

                     Class: setosa Class: versicolor Class: virginica
Sensitivity                 1.0000            0.9333           0.8667
Specificity                 1.0000            0.9333           0.9667
Pos Pred Value              1.0000            0.8750           0.9286
Neg Pred Value              1.0000            0.9655  